# Final Model

### Import Packages

In [1]:
import os
import numpy as np
import pandas as pd
import csv
from scipy import stats
from scipy.special import inv_boxcox
import sklearn.metrics as met
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.linear_model import Lasso, Ridge, LinearRegression, LogisticRegression
from sklearn.kernel_ridge import KernelRidge

import itertools
import multiprocessing
import matplotlib.pyplot as plt

from datetime import datetime

from numpy.linalg import inv
from sklearn.utils import shuffle

import timeit

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
print(tf.__version__)

Using TensorFlow backend.


2.0.0


In [2]:
#expand dataframe viewing capability
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Import Data

In [3]:
PTNdata_masterPath = ('/Users/nickcforrest/Documents/My Documents/Education/AFIT/AFIT WORK/Thesis Work/Python Files/PTNdata1_master.csv')
PTNdata_master = pd.read_csv(PTNdata_masterPath).iloc[0:, 1:]
PTNdata_master.head()

,Student ID,Training Date,Mission Analysis/Products,Ground Ops,Takeoff,Departure,Basic Aircraft Control,Cross-Check,Enroute Descent / Recovery,Inflight Checks,Inflight Planning,Clearing / Visual Lookout,Communication,Risk Mgmt / Decision Making,Situational Awareness,Task Management,Emergency Procedures,General Knowledge,Overhead/Closed Pattern,Visual St-In,Landing,No-Flap Landing,Go-Around,Emergency Landing Pattern,G-Awareness,TP Stalls,Slow Flight,Power On Stalls,Contact Recoveries,Spin Recovery,Aileron Roll,Barrel Roll,Pitchback / Sliceback,Cloverleaf,Cuban Eight,Immelmann,Lazy Eight,Loop,Split S,Vertical S,Unusual Attitudes,Steep Turns,Intercept / Maintain Arc,Fix to Fix,Holding,Full Procedure Approach,Non-Precision Final,Precision Final,Circling Approach,Missed Approach,Night Landing,Wing Takeoff,Interval Takeoff,Instrument Trail,G-Warmup / Awareness,Lead Platform,Pitchout (Both),Fingertip (Wing),Route (Wing),Fighting Wing (Wing),Straight Ahead Rejoin,Turning Rejoin,Overshoot,Echelon (Wing),Breakout (Wing),Lost Wingman (Both),Extended Trail (Wing),Position Change,Formation Approach (Both),Formation Landing (Both),Battle Damage Check,Flt Integrity / Wingman Consideration,Delay 90,Delay 45,Hook Turn,Shackle,Cross Turn,Fluid Turn,Tactical Rejoins,Fluid Maneuvering,Tac Initial,Course Mx,Course Entry,Time Control,Altitude Control,Checkpoint ID,LL GPS Integration,Tactical Maneuvering,LL Lead Change,Four Ship Admin,Fluid 4,Box Formation,Offset Box,Wall,4-Ship Fingertip,4-Ship Straight Ahead Rejoin,4-Ship Turning Rejoin,Heat to Guns Setup,Heat to Guns Maneuvering,Fuel Awareness/Management,Advanced Handling,Perch Setups,Maneuver Selection,Offensive Fighter Mnvr Exec,Defensive Fighter Mnvr Exec,CZ Recognition,Air to Air Weapons Employ,HA Lead Turn Exercise,HA Butterfly Setups,HA BFM Flt Analysis,SA Conventional Range,SA Tactical Range Proc,SA Safe-Excape Maneuver,SA Threat Reaction,SA Weapons Employment,Air to Ground Error Analysis,TACS/JFIRE Procedures,Air to Gnd 2-Ship Mutual Supt,Mission Management,VFR Arrival,Tanker Procedures,Reciever Procedures,Airdrop Procedures,Crew Coordination,Single Engine Approach,Single Engine GA/Missed Appch,A/R Overrun,A/R Breakaway,FD/AP Operations,FMS Operations,Device,Training Exercise,Total MIF,FPS
0,1,2018-04-02,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,S,1,36,44.25
1,1,2018-04-03,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,S,2,49,57.75
2,1,2018-04-04,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,3,57,67.50
3,1,2018-04-06,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,0.0,3.0,0.0,0.0,0.0,2.0,2.0,2

In [4]:
numManeuvers  = PTNdata_master.iloc[:,2:-4].shape[1]
numManeuvers

128

##### Categories Seen and Categories MIF Lists

In [5]:
#put maneuvers into categories
basicManeuverNames = ['Mission Analysis/Products', 'Ground Ops','Takeoff','Departure','Basic Aircraft Control','Cross-Check','Enroute Descent / Recovery','Inflight Checks','Inflight Planning','Clearing / Visual Lookout','Communication','Risk Mgmt / Decision Making','Situational Awareness','Task Management','Emergency Procedures','General Knowledge']
patternsNames = ['Overhead/Closed Pattern', 'Visual St-In', 'Landing', 'No-Flap Landing', 'Go-Around', 'Emergency Landing Pattern']
contactNames = ['G-Awareness', 'TP Stalls', 'Slow Flight', 'Power On Stalls', 'Contact Recoveries', 'Spin Recovery', 'Aileron Roll', 'Barrel Roll', 'Pitchback / Sliceback', 'Cloverleaf', 'Cuban Eight', 'Immelmann', 'Lazy Eight', 'Loop', 'Split S']
instrumentNames =['Vertical S', 'Unusual Attitudes', 'Steep Turns', 'Intercept / Maintain Arc', 'Fix to Fix', 'Holding', 'Full Procedure Approach', 'Non-Precision Final', 'Precision Final', 'Circling Approach', 'Missed Approach', 'Night Landing']
basicFormationNames = ['Wing Takeoff', 'Interval Takeoff', 'Instrument Trail', 'G-Warmup / Awareness', 'Lead Platform', 'Pitchout (Both)', 'Fingertip (Wing)', 'Route (Wing)', 'Fighting Wing (Wing)', 'Straight Ahead Rejoin', 'Turning Rejoin', 'Overshoot', 'Echelon (Wing)', 'Breakout (Wing)', 'Lost Wingman (Both)', 'Extended Trail (Wing)', 'Position Change', 'Formation Approach (Both)', 'Formation Landing (Both)', 'Battle Damage Check', 'Flt Integrity / Wingman Consideration']
tacticalFormationNames =['Delay 90', 'Delay 45', 'Hook Turn', 'Shackle', 'Cross Turn', 'Fluid Turn', 'Tactical Rejoins', 'Fluid Maneuvering', 'Tac Initial']
lowLevelNames = ['Course Mx', 'Course Entry', 'Time Control', 'Altitude Control', 'Checkpoint ID', 'LL GPS Integration', 'Tactical Maneuvering', 'LL Lead Change']
fourShipFormationNames = ['Four Ship Admin', 'Fluid 4', 'Box Formation', 'Offset Box', 'Wall', '4-Ship Fingertip','4-Ship Straight Ahead Rejoin','4-Ship Turning Rejoin']
cafIntroNames = ['Heat to Guns Setup', 'Heat to Guns Maneuvering ', 'Fuel Awareness/Management', 'Advanced Handling', 'Perch Setups', 'Maneuver Selection', 'Offensive Fighter Mnvr Exec', 'Defensive Fighter Mnvr Exec', 'CZ Recognition', 'Air to Air Weapons Employ', 'HA Lead Turn Exercise', 'HA Butterfly Setups', 'HA BFM Flt Analysis', 'SA Conventional Range', 'SA Tactical Range Proc', 'SA Safe-Excape Maneuver', 'SA Threat Reaction', 'SA Weapons Employment', 'Air to Ground Error Analysis', 'TACS/JFIRE Procedures', 'Air to Gnd 2-Ship Mutual Supt']
mafIntroNames = ['Mission Management', 'VFR Arrival', 'Tanker Procedures', 'Reciever Procedures', 'Airdrop Procedures', 'Crew Coordination', 'Single Engine Approach', 'Single Engine GA/Missed Appch', 'A/R Overrun', 'A/R Breakaway', 'FD/AP Operations', 'FMS Operations']

In [6]:
#list categories of flight maneuvers
categoryList = ['basicManeuver','patterns','contact','instrument','basicFormation','tacticalFormation','lowLevel','fourShipFormation','cafIntro','mafIntro']

In [7]:
#define lengths of all categories
basicManeuverLength = len(basicManeuverNames)
patternsLength = len(patternsNames)
contactLength = len(contactNames)
instrumentLength = len(instrumentNames)
basicFormationLength = len(basicFormationNames)
tacticalFormationLength = len(tacticalFormationNames)
lowLevelLength = len(lowLevelNames)
fourShipFormationLength = len(fourShipFormationNames)
cafIntroLength = len(cafIntroNames)
mafIntroLength = len(mafIntroNames)

#make sure the sum includes all maneuvers
print([basicManeuverLength,patternsLength,contactLength,instrumentLength,basicFormationLength,tacticalFormationLength,lowLevelLength,fourShipFormationLength,cafIntroLength,mafIntroLength])

[16, 6, 15, 12, 21, 9, 8, 8, 21, 12]


In [8]:
firstManCol = 2 #first maneuver column is in column 3

#define column ranges for each maneuver category
basicManeuverRange = range(firstManCol,firstManCol + basicManeuverLength)
patternsRange = range(max(basicManeuverRange)+1,max(basicManeuverRange) + patternsLength)
contactRange = range(max(patternsRange)+1,max(patternsRange) + contactLength)
instrumentRange = range(max(contactRange)+1,max(contactRange)+instrumentLength)
basicFormationRange = range(max(instrumentRange)+1,max(instrumentRange)+basicFormationLength)
tacticalFormationRange = range(max(basicFormationRange)+1,max(basicFormationRange)+tacticalFormationLength)
lowLevelRange = range(max(tacticalFormationRange)+1,max(tacticalFormationRange)+lowLevelLength)
fourShipFormationRange = range(max(lowLevelRange)+1,max(lowLevelRange)+fourShipFormationLength)
cafIntroRange = range(max(fourShipFormationRange)+1,max(fourShipFormationRange)+cafIntroLength)
mafIntroRange = range(max(cafIntroRange)+1,max(cafIntroRange)+mafIntroLength)

#make sure ranges line up
print(basicManeuverRange,patternsRange,contactRange,instrumentRange,basicFormationRange,tacticalFormationRange,lowLevelRange,fourShipFormationRange,cafIntroRange,mafIntroRange)

range(2, 18) range(18, 23) range(23, 37) range(37, 48) range(48, 68) range(68, 76) range(76, 83) range(83, 90) range(90, 110) range(110, 121)


In [9]:
#define dictionary for value look up of column ranges

rangeDict = dict({
    
'RangebasicManeuver' : [min(basicManeuverRange),max(basicManeuverRange)+1],
'Rangepatterns' : [min(patternsRange),max(patternsRange)+1],
'Rangecontact' : [min(contactRange),max(contactRange)+1],
'Rangeinstrument'  : [min(instrumentRange),max(instrumentRange)+1],
'RangebasicFormation'  : [min(basicFormationRange),max(basicFormationRange)+1],
'RangetacticalFormation'  : [min(tacticalFormationRange),max(tacticalFormationRange)+1],
'RangelowLevel'  : [min(lowLevelRange),max(lowLevelRange)+1],
'RangefourShipFormation'  : [min(fourShipFormationRange),max(fourShipFormationRange)+1],
'RangecafIntro' : [min(cafIntroRange),max(cafIntroRange)+1],
'RangemafIntro'  : [min(mafIntroRange),max(mafIntroRange)+1],
})


##### Make scores a fraction of individual MIF for each maneuver to more appropriately track progress

In [10]:
#list of mifs in order
maneuverMIF = [4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,3,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,3,3,4,4,4,4,4,3,4,4,4,3,3,4,4,4,4,4,4,4,3,4,3,3,3,3,4,3]

#make df to reference 
maneuverMIF_df = pd.DataFrame([PTNdata_master.iloc[:,2:-4].columns,maneuverMIF])
maneuverMIF_df.columns = maneuverMIF_df.iloc[0]
maneuverMIF_df.drop(maneuverMIF_df.index[0],inplace=True)
maneuverMIF_df


,Mission Analysis/Products,Ground Ops,Takeoff,Departure,Basic Aircraft Control,Cross-Check,Enroute Descent / Recovery,Inflight Checks,Inflight Planning,Clearing / Visual Lookout,Communication,Risk Mgmt / Decision Making,Situational Awareness,Task Management,Emergency Procedures,General Knowledge,Overhead/Closed Pattern,Visual St-In,Landing,No-Flap Landing,Go-Around,Emergency Landing Pattern,G-Awareness,TP Stalls,Slow Flight,Power On Stalls,Contact Recoveries,Spin Recovery,Aileron Roll,Barrel Roll,Pitchback / Sliceback,Cloverleaf,Cuban Eight,Immelmann,Lazy Eight,Loop,Split S,Vertical S,Unusual Attitudes,Steep Turns,Intercept / Maintain Arc,Fix to Fix,Holding,Full Procedure Approach,Non-Precision Final,Precision Final,Circling Approach,Missed Approach,Night Landing,Wing Takeoff,Interval Takeoff,Instrument Trail,G-Warmup / Awareness,Lead Platform,Pitchout (Both),Fingertip (Wing),Route (Wing),Fighting Wing (Wing),Straight Ahead Rejoin,Turning Rejoin,Overshoot,Echelon (Wing),Breakout (Wing),Lost Wingman (Both),Extended Trail (Wing),Position Change,Formation Approach (Both),Formation Landing (Both),Battle Damage Check,Flt Integrity / Wingman Consideration,Delay 90,Delay 45,Hook Turn,Shackle,Cross Turn,Fluid Turn,Tactical Rejoins,Fluid Maneuvering,Tac Initial,Course Mx,Course Entry,Time Control,Altitude Control,Checkpoint ID,LL GPS Integration,Tactical Maneuvering,LL Lead Change,Four Ship Admin,Fluid 4,Box Formation,Offset Box,Wall,4-Ship Fingertip,4-Ship Straight Ahead Rejoin,4-Ship Turning Rejoin,Heat to Guns Setup,Heat to Guns Maneuvering,Fuel Awareness/Management,Advanced Handling,Perch Setups,Maneuver Selection,Offensive Fighter Mnvr Exec,Defensive Fighter Mnvr Exec,CZ Recognition,Air to Air Weapons Employ,HA Lead Turn Exercise,HA Butterfly Setups,HA BFM Flt Analysis,SA Conventional Range,SA Tactical Range Proc,SA Safe-Excape Maneuver,SA Threat Reaction,SA Weapons Employment,Air to Ground Error Analysis,TACS/JFIRE Procedures,Air to Gnd 2-Ship Mutual Supt,Mission Management,VFR Arrival,Tanker Procedures,Reciever Procedures,Airdrop Procedures,Crew Coordination,Single Engine Approach,Single Engine GA/Missed Appch,A/R Overrun,A/R Breakaway,FD/AP Operations,FMS Operations
1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,3,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,3,3,4,4,4,4,4,3,4,4,4,3,3,4,4,4,4,4,4,4,3,4,3,3,3,3,4,3


In [11]:
ALL_MIF_RATIOS = False

In [12]:
#convert to fraction over individual mifs
if ALL_MIF_RATIOS:
    PTNdata = PTNdata_master
    PTNdata.iloc[:,2:-4] = PTNdata_master.iloc[:,2:-4]/maneuverMIF
    PTNdata.head()
else: 
    PTNdata_mifPath = ('/Users/nickcforrest/Documents/My Documents/Education/AFIT/AFIT WORK/Thesis Work/Python Files/PTNdata1_mif.csv')
    PTNdata = pd.read_csv(PTNdata_mifPath).iloc[0:, 1:]
    


### Set Forward Progress Score(FPS) Parameters

In [13]:
#set weights for each category
basicManeuverWeight = 6
patternsWeight = 6
contactWeight = 6
instrumentWeight = 6
basicFormationWeight = 6
tacticalFormationWeight = 6
lowLevelWeight = 6
fourShipFormationWeight = 6
cafIntroWeight = 6
mafIntroWeight = 6

In [14]:
#define weight lists
categoryMIFweight = [basicManeuverWeight,patternsWeight,contactWeight,instrumentWeight,basicFormationWeight,tacticalFormationWeight,lowLevelWeight,fourShipFormationWeight,cafIntroWeight,mafIntroWeight]
categorySeenweight = [(x-3) for x in [basicManeuverWeight,patternsWeight,contactWeight,instrumentWeight,basicFormationWeight,tacticalFormationWeight,lowLevelWeight,fourShipFormationWeight,cafIntroWeight,mafIntroWeight]]
#maneuverMIFweight = 2.25 delete me
#maneuverSeenweight = 3 delete me

# USER INFORMATION

### Set User Parameters

##### Enter User's ID Number:

In [32]:
#make sure this is in the same data type as the data
userID = 3

In [33]:
#error message that userID must be a string 
#THIS NEEDS TO BE ADJUST BASED ON REAL USER IDs AND NOT ARBITRARILY ASSIGNED NUMBERS
assert userID == int(userID), "ERROR: double check that you entered the USER ID properly!"

##### Enter Training Exercise Number That Flight Maneuver Recomendation Is For:

In [28]:
TE = 55

In [29]:
#error message that userID must be a string 
assert TE == int(TE), "ERROR: The training exercise must be a whole number!"

### User's MIF Ratio List and Category Progress From All Grades Before The Requested Training Exercise 

In [34]:
#define start time of recommedenation
recommendTime_start= timeit.default_timer()

In [35]:
#user's data so far
userData = PTNdata[PTNdata['Student ID']== userID].iloc[0:TE,2:-4]
userData

,Mission Analysis/Products,Ground Ops,Takeoff,Departure,Basic Aircraft Control,Cross-Check,Enroute Descent / Recovery,Inflight Checks,Inflight Planning,Clearing / Visual Lookout,Communication,Risk Mgmt / Decision Making,Situational Awareness,Task Management,Emergency Procedures,General Knowledge,Overhead/Closed Pattern,Visual St-In,Landing,No-Flap Landing,Go-Around,Emergency Landing Pattern,G-Awareness,TP Stalls,Slow Flight,Power On Stalls,Contact Recoveries,Spin Recovery,Aileron Roll,Barrel Roll,Pitchback / Sliceback,Cloverleaf,Cuban Eight,Immelmann,Lazy Eight,Loop,Split S,Vertical S,Unusual Attitudes,Steep Turns,Intercept / Maintain Arc,Fix to Fix,Holding,Full Procedure Approach,Non-Precision Final,Precision Final,Circling Approach,Missed Approach,Night Landing,Wing Takeoff,Interval Takeoff,Instrument Trail,G-Warmup / Awareness,Lead Platform,Pitchout (Both),Fingertip (Wing),Route (Wing),Fighting Wing (Wing),Straight Ahead Rejoin,Turning Rejoin,Overshoot,Echelon (Wing),Breakout (Wing),Lost Wingman (Both),Extended Trail (Wing),Position Change,Formation Approach (Both),Formation Landing (Both),Battle Damage Check,Flt Integrity / Wingman Consideration,Delay 90,Delay 45,Hook Turn,Shackle,Cross Turn,Fluid Turn,Tactical Rejoins,Fluid Maneuvering,Tac Initial,Course Mx,Course Entry,Time Control,Altitude Control,Checkpoint ID,LL GPS Integration,Tactical Maneuvering,LL Lead Change,Four Ship Admin,Fluid 4,Box Formation,Offset Box,Wall,4-Ship Fingertip,4-Ship Straight Ahead Rejoin,4-Ship Turning Rejoin,Heat to Guns Setup,Heat to Guns Maneuvering,Fuel Awareness/Management,Advanced Handling,Perch Setups,Maneuver Selection,Offensive Fighter Mnvr Exec,Defensive Fighter Mnvr Exec,CZ Recognition,Air to Air Weapons Employ,HA Lead Turn Exercise,HA Butterfly Setups,HA BFM Flt Analysis,SA Conventional Range,SA Tactical Range Proc,SA Safe-Excape Maneuver,SA Threat Reaction,SA Weapons Employment,Air to Ground Error Analysis,TACS/JFIRE Procedures,Air to Gnd 2-Ship Mutual Supt,Mission Management,VFR Arrival,Tanker Procedures,Reciever Procedures,Airdrop Procedures,Crew Coordination,Single Engine Approach,Single Engine GA/Missed Appch,A/R Overrun,A/R Breakaway,FD/AP Operations,FMS Operations
170,0.75,0.50,0.75,0.75,0.50,0.50,0.75,0.50,0.75,0.75,0.50,0.25,0.25,0.25,0.50,0.50,0.50,0.00,0.50,0.0,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,0.75,0.50,0.50,0.50,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.25,0.25,0.50,0.50,0.00,0.00,0.50,0.0,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.75,0.50,0.25,0.00,0.000000,0.00,0.00,0.00,0.25,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172,0.50,0.50,0.50,0.25,0.50,0.25,0.25,0.50,0.50,0.25,0.50,0.25,0.25,0.50,0.50,0.50,0.50,0.00,0.50,0.0,0.00,0.000000,0.50,0.50,0.00,0.50,0.00,0.000000,0.75,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.50,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.50,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

##### MIF Ratio List

In [ ]:
#list of max ratio for each value
userMIFRatios = [0]*numManeuvers

#set items in list equal to the max ratio they have received up to the specified point in training thus far.
#using -2 because index starts at 0 and we want up to TE-1
for manNum,manScore in enumerate(userData):
        userMIFRatios[manNum] = userData[manScore][0:TE-2].max()

print(userMIFRatios)

In [ ]:
#function to check if all values in a list are equal to a given value
def check(mylist, val): 
    return(all(x >= val for x in mylist)) 

In [ ]:
#define list for categories seen and categories MIF
userCatSeen = [0]*len(categoryList)
userCatMIF = [0]*len(categoryList)   

#boolean for categories in terms of maneuvers
for catNum, catName in enumerate(categoryList):

#boolean if all maneuvers in a category have been seen, in terms of maneuvers 
    currentRange = 'Range%s' % catName

    if 0 in userMIFRatios[rangeDict[currentRange][0] - firstManCol:rangeDict[currentRange][1] - firstManCol]:
        userCatSeen[catNum] = 0
    else:
        userCatSeen[catNum] = 1

#boolean if all maneuvers in a category have maxed mif, in terms of maneuvers
    if (check(userMIFRatios[rangeDict[currentRange][0] - firstManCol:rangeDict[currentRange][1] - firstManCol],1)):
        userCatMIF[catNum] = 1
    else:
        userCatMIF[catNum] = 0
        
print('Categories Seen',userCatSeen)
print()
print('Categories MIF',userCatMIF)

### User's current FPS Score

In [ ]:
#calculate FPS score
userFPS = sum([categoryMIFweight[x]*userCatMIF[x] for x in range(len(userCatMIF))]) + sum([categorySeenweight[x]*userCatSeen[x] for x in range(len(userCatSeen))]) + sum(userMIFRatios)
print(userFPS)        

# GLOBAL INFORMATION

### Global Golden Standard MIF Ratio List and Category Progress From All Grades Before The Requested Training Exercise 

##### Define Global Data Excluding User's Data

In [ ]:
#global data so far
globalData = PTNdata[(PTNdata['Student ID']!= userID) & (PTNdata['Training Exercise'] < TE)]
globalData.head()

##### Identifying Students From History in Top 10% of Global Data When at Same Training Exercise as User

In [ ]:
#historic FPS scores at this point
maxTE_all = globalData[globalData['Training Exercise'] == TE-1]
maxTE_all

totNumStuds = len(globalData['Student ID'].unique())
print(totNumStuds)

In [ ]:
#list percentage used for golden standard
gs_perc = .10 

In [ ]:
#identify number of students in top 10% of all students
gs_numStuds = round(gs_perc*totNumStuds)
gs_numStuds

In [ ]:
gs_studs = list(maxTE_all.nlargest(gs_numStuds,['FPS'])['Student ID'].unique())
gs_studs

##### Make MIF Ratio List for all students in gold Standard

In [ ]:
#initialize list of max ratio for each value
gs_totMIFRatios = np.zeros((len(gs_studs),numManeuvers))

j = 0
for i in gs_studs:
    #set items in list equal to the max ratio they have received up to the specified point in training thus far.
    for manNum,manScore in enumerate(globalData[globalData['Student ID']== i].iloc[:,2:-4]):
        gs_totMIFRatios[j][manNum] = globalData[manScore].max()
    j+=1
#print(gs_totMIFRatios)   

##### Global Standard MIF Ratio List Based on Top 10% Averages

In [ ]:
#average all top 10% MIf Ratio Lists to create golden standard
#gs_MIFRatios = [0]*numManeuvers

gs_MIFRatios = gs_totMIFRatios.mean(axis=0)

#print(gs_MIFRatios)

##### Global Standard Category Progress List Based on Top 10% Averages

NOTE: worth looking into how the averages lines up with the points system. 

In [ ]:
#define list for categories seen and categories MIF
gs_CatSeen = [0]*len(categoryList)
gs_CatMIF = [0]*len(categoryList)    

#boolean for categories in terms of maneuvers
for catNum, catName in enumerate(categoryList):
    
#boolean if all maneuvers in a category have been seen, in terms of maneuvers 
    currentRange = 'Range%s' % catName

    if 0 in gs_MIFRatios[rangeDict[currentRange][0] - firstManCol:rangeDict[currentRange][1] - firstManCol]:
        gs_CatSeen[catNum] = 0
    else:
        gs_CatSeen[catNum] = 1
#boolean if all maneuvers in a category have maxed mif, in terms of maneuvers
    if (check(gs_MIFRatios[rangeDict[currentRange][0] - firstManCol:rangeDict[currentRange][1] - firstManCol],1)):
        gs_CatMIF[catNum] = 1
    else:
        gs_CatMIF[catNum] = 0
        
print('Categories Seen',gs_CatSeen)
print()
print('Categories MIF',gs_CatMIF)

### Global Standard FPS Score

In [ ]:
#calculate FPS score
gs_FPS = sum([categoryMIFweight[x]*gs_CatMIF[x] for x in range(len(gs_CatMIF))]) + sum([categorySeenweight[x]* gs_CatSeen[x] for x in range(len(gs_CatSeen))]) + sum(gs_MIFRatios)
print(gs_FPS)        

##### Difference Between GS MIF Ratios and User MIF Ratios
positive numbers indicate a performance that is below the golden standard. negative numbers represent a performance that is above the golden standard.

In [ ]:
difMIFRatios = gs_MIFRatios-userMIFRatios
difMIFRatios

### Establish List of probabilities for each exercise occuring on this training exercise

In [ ]:
# how many training days each individual student had
exercise_per_student = pd.DataFrame(PTNdata_master['Student ID'].value_counts())
exercise_per_student = exercise_per_student.sort_values(by = ['Student ID'])

#calculate the max length of training so far in training exercises. 
maxExercises = exercise_per_student['Student ID'].max()

#calculate sample size at each training exercise
student_per_exerciseNum = PTNdata_master['Training Exercise'].value_counts().sort_index().values
print(student_per_exerciseNum)

In [ ]:
#create variables with frequency for every maneuver for every training exercise number for all records but current user.  
#(including MAF and CAF tracks for now)
#can use master data set here because this student would not be encorporated into the global set yet if they are receiving the recommendation. 
num_of_rows = len(PTNdata_master.iloc[0,2:-4])
num_of_cols = maxExercises

globalManeuverCounts = np.zeros([num_of_rows, num_of_cols])
print(np.shape(globalManeuverCounts))

#populate maneuver counts and tells you how many times each manuever occured on each training day. 
i = 1
j = 1
position_row = j-1
position_col = i-1

       
for col in PTNdata_master.iloc[:,2:-4].columns:
    for numEval in PTNdata_master['Training Exercise'].unique():
        notNulls = [bool(x>0) for x in PTNdata_master[PTNdata_master['Training Exercise']==numEval][col]] #whether or not maneuver was performed.
        values = [sum(notNulls)] #sum up all maneuvers performed on training day
        #print(numEval,'---',col,'---',values)
        position = [int(num_of_cols * position_row + i-1)] #put count in data frame

        np.put(globalManeuverCounts, position, values)
        i = i+1

    j = j+1


In [ ]:
#label columns and index
globalManeuverCounts = pd.DataFrame(globalManeuverCounts)
globalManeuverCounts.columns = [np.arange(1, maxExercises+1)]
globalManeuverCounts.index = PTNdata_master.iloc[:,2:-4].columns
globalManeuverCounts.head()

In [ ]:
#Export maneuver counts to CSV file 
globalManeuverCounts.to_csv(r'/Users/nickcforrest/Documents/My Documents/Education/AFIT/AFIT WORK/Thesis Work/Python Files/globablManeuverCounts.csv')

In [ ]:
#Divide columns of maneuver counts by # of Students still in training at each exercise. 
#This tells us the fraction of active students that performed maneuver on that day.

globalManeuverPerc = globalManeuverCounts / student_per_exerciseNum
globalManeuverPerc.head()

In [ ]:
#Export maneuver percentages to CSV file 
globalManeuverPerc.to_csv(r'/Users/nickcforrest/Documents/My Documents/Education/AFIT/AFIT WORK/Thesis Work/Python Files/globalManeuverPerc.csv')

In [ ]:


#list of probabilities of occurance given training day. 
pOccurList = list(globalManeuverPerc.iloc[:,TE-1])
print(pOccurList)

### Calculate Utility of Recommending Each Maneuver on This Training Day

In [ ]:
manUtils = [difMIFRatios[x]*pOccurList[x] for x in range(len(difMIFRatios))]
print(manUtils)

In [ ]:
#(maneuver index, utility value)
meta_manUtils = list(enumerate(manUtils))
print(meta_manUtils)

In [ ]:
#sort utilities
#sort top down
def Sort(sub_li): 
    return(sorted(sub_li, key = lambda x: x[1],reverse=True))


In [ ]:
meta_manUtils = Sort(meta_manUtils)
print(meta_manUtils)

In [ ]:
#function to check if all utility values are 0 or less.
def checkNegative(mylist, val): 
    return(all(x < val for x in mylist),sum([1 for x in mylist if x > 0]))

In [ ]:
#set value to .00001 because we want to include 0.0 since that means a student is equal to the standard. 
#you can't be above an in indiv maneuver standard if the maneuver standard is at 4 points. 

if checkNegative(manUtils,0.00001)[0]:
    ABOVE_GS = False
else:
    ABOVE_GS = True
    
#number of events under the golden standard
numBelow = checkNegative(manUtils,0.00001)[1]
print(numBelow)

In [ ]:
print(ABOVE_GS)

### Apply LSTM Model for Recommendation

### Import Deep Learning Packages

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#create input list of lists containing scores up to and at each particular training exercise
#dont include last record because there is no target variable definable for last record per student
#userInput_raw = [userData.values.tolist() for i in range(1,len(userData.index))]
userInput_raw = [userData.values.tolist()]
#check type to make sure its a list of lists
#print(userInput_raw[0])
      
#pad sequences using pre-sequence truncation
#when over 50 training exercises are in memory bank, remove earliest timesteps from the beginning of sequences
seq_length = 50
userInput = pad_sequences(userInput_raw[-seq_length:], padding='pre',dtype='float32',maxlen = seq_length)
#print(userInput)

In [ ]:
# load model from ModelTraining_PTNdata1 file 
model = load_model('final_model.h5')

In [ ]:
# make raw prediction
pred = model.predict(userInput, verbose=0)
print(pred)

In [ ]:
#define function that rounds raw estimations into boolean outputs
split_thresh = 0.5
def toBoolean(mylist):
    l = [[1 if y >= split_thresh else 0 for y in x] for i,x in enumerate(mylist)]
    return l

In [ ]:
#round prediction to recomendation
recom = toBoolean(pred)[0]
print(recom)

### Calculate Projected FPS Increase Given Content-Based Recomendation

In [ ]:
#convert mif scores to original grade scores
userMIFscore = [userMIFRatios[x]*maneuverMIF[x] for x in range(len(maneuverMIF))]

#add projected points from recommended maneuvers
userMIFscore_proj = [userMIFscore[a]+recom[a] if userMIFscore[a]<4 else userMIFscore[a] for a in range(len(maneuverMIF))]

#convert back to projected mif ratio
userMIFRatios_proj = [userMIFscore_proj[x]/maneuverMIF[x] for x in range(len(maneuverMIF))]
print(userMIFRatios_proj)


In [ ]:
#define list for categories seen and categories MIF
userCatSeen_proj = [0]*len(categoryList)
userCatMIF_proj = [0]*len(categoryList)   

#boolean for categories in terms of maneuvers
for catNum, catName in enumerate(categoryList):

#boolean if all maneuvers in a category have been seen, in terms of maneuvers 
    currentRange = 'Range%s' % catName

    if 0 in userMIFRatios_proj[rangeDict[currentRange][0] - firstManCol:rangeDict[currentRange][1] - firstManCol]:
        userCatSeen_proj[catNum] = 0
    else:
        userCatSeen_proj[catNum] = 1

#boolean if all maneuvers in a category have maxed mif, in terms of maneuvers
    if (check(userMIFRatios_proj[rangeDict[currentRange][0] - firstManCol:rangeDict[currentRange][1] - firstManCol],1)):
        userCatMIF_proj[catNum] = 1
    else:
        userCatMIF_proj[catNum] = 0
        
print('Categories Seen',userCatSeen_proj)
print()
print('Categories MIF',userCatMIF_proj)

In [ ]:
#calculate projected FPS score
userFPS_proj = sum([categoryMIFweight[x]*userCatMIF_proj[x] for x in range(len(userCatMIF_proj))]) + sum([categorySeenweight[x]* userCatSeen_proj[x] for x in range(len(userCatSeen_proj))]) + sum(userMIFRatios_proj)
print(userFPS_proj)        

### If User is Below Golden Standard...

In [ ]:
#ABOVE_GS = True
#sets number of swaps you want to make to LSTM recommendation
if numBelow <= 5:
    thresh = 1
elif numBelow in range(6,11):
    thresh = 2
elif numBelow in range(11,21):
    thresh = 3
elif numBelow in range(21,31):
    thresh = 4
elif numBelow in range(31,41):
    thresh = 5
else: 
    thresh = 6
    

In [ ]:
if ABOVE_GS == False: 
    #print('LSTM recomendation is:', recom) 
    i = 0
    j = 0
    for x in reversed(meta_manUtils):
        if recom[x[0]] == 1:
            print('least util maneuver is',maneuverMIF_df.columns[x[0]],'with util value',x[1])
            recom[x[0]] = 0
            i+=1
            if i == thresh: break

    for y in meta_manUtils:
        if recom[y[0]] == 0:
            print('most util maneuver is',maneuverMIF_df.columns[y[0]],'with util value',y[1])
            recom[y[0]] = 1
            j+=1
            if j == thresh: break      
                
print(recom)

### If User is Above Golden Standard...

In [ ]:

if ABOVE_GS:
    poss = recom
    all_poss = [recom]
    
    #define all possible maneuver swaps, only swapping maneuvers for those with better utility.
    for index,y in enumerate(meta_manUtils):

        #loop through maneuvers with utility > 0
        #print(index,y)
        if y[1] > 0:
            #if maneuver is not recommended, recommend it.
            if poss[y[0]] == 0: 
                poss[y[0]] = 1
                
                #get rid of each individual maneuver that has let utility and add new recom to possible recom list.
                for indy,x in enumerate(meta_manUtils[index + 1:]):
                    #print(x)
                    if poss[x[0]] == 1:
                        poss[x[0]] = 0

                        #add another possible maneuver swap combo to the list 
                        #must be poss[:] here because otherwise youll continue to update every element in list as newest element
                        all_poss.append(poss[:])
                        poss[x[0]] = 1
                        
                poss[y[0]] = 0

    #Calculate expected FPS for each element in all_poss

    #list of FPS scores for each combo 
    FPS_poss = []

    #define current users MIFscore
    userMIFscore = [userMIFRatios[x]*maneuverMIF[x] for x in range(len(maneuverMIF))]

    for combo in all_poss:
        #print(combo)
        #add projected points from recommended maneuvers
        comboMIFscore_proj = [userMIFscore[a]+combo[a] if userMIFscore[a]<4 else userMIFscore[a] for a in range(len(maneuverMIF))]
        
        #convert back to projected mif ratio
        comboMIFRatios_proj = [comboMIFscore_proj[x]/maneuverMIF[x] for x in range(len(maneuverMIF))]
         
        #list of all points awarded
        manScorePoints = [0]*numManeuvers
        
        for manNum,manScore in enumerate(comboMIFRatios_proj):
            #maneuver has reached max mif AKA proficiency. adds 2.25 for improving.
            if manScore>=1:
                manScorePoints[manNum] = 5
            #maneuver has been seen but max grade 1. adds 1.5 for improving Scores in thirds to account for maneuvers with MIF of 3. 
            elif (manScore<=(1/3) and manScore>0):
                manScorePoints[manNum] = 1.5
            #maneuver ha been seen and max grade 2. adds 0.75 for improving.
            elif (manScore<(2/3) and manScore>(1/3)):
                manScorePoints[manNum] = 2.25
            #maneuver has been seen and max grade 3. adds 0.5 for improving.
            elif (manScore<1 and manScore>=(2/3)):
                manScorePoints[manNum] = 2.75
        
        #define list for categories seen and categories MIF
        comboCatSeen_proj = [0]*len(categoryList)
        comboCatMIF_proj = [0]*len(categoryList)   

        #boolean for categories in terms of maneuvers
        for catNum, catName in enumerate(categoryList):

        #boolean if all maneuvers in a category have been seen, in terms of maneuvers 
            currentRange = 'Range%s' % catName

            if 0 in comboMIFRatios_proj[rangeDict[currentRange][0] - firstManCol:rangeDict[currentRange][1] - firstManCol]:
                comboCatSeen_proj[catNum] = 0
            else:
                comboCatSeen_proj[catNum] = 1

        #boolean if all maneuvers in a category have maxed mif, in terms of maneuvers
            if (check(comboMIFRatios_proj[rangeDict[currentRange][0] - firstManCol:rangeDict[currentRange][1] - firstManCol],1)):
                comboCatMIF_proj[catNum] = 1
            else:
                comboCatMIF_proj[catNum] = 0
                
        #calculate projected FPS score
        comboFPS_proj = sum([categoryMIFweight[x]*comboCatMIF_proj[x] for x in range(len(comboCatMIF_proj))]) + sum([categorySeenweight[x]* comboCatSeen_proj[x] for x in range(len(comboCatSeen_proj))]) + sum(manScorePoints)    
        #print(comboFPS_proj) 
        #add combo score to list of all possible improving scores    
        FPS_poss.append(comboFPS_proj)

    #identify the max projected FPS score 
    list_max = max(FPS_poss)
    print('max projected FPS:',list_max)

    #identify all possible combos to get projected max. 
    list_index = [i for i, j in enumerate(FPS_poss) if j == list_max]
    print('possible combo indicies:',list_index)

    #randomly select one of the best options
    import random
    recom_index = random.choice(list_index)
    print('idex of random best combo:',recom_index)

    recom = all_poss[recom_index]
    print('recommendation:',recom)

### Convert to maneuver names for recommendation

#### Extra Maneuvers

In [ ]:
extra = recom
print(extra)
#create list of extra maneuver utilities
extraUtils = [0]*numManeuvers
for x in meta_manUtils:
    if extra[x[0]] == 1:
        extraUtils[x[0]] = 999
    if extra[x[0]] == 0:
        extraUtils[x[0]] = x[1]
extraUtils = list(filter((999).__ne__, extraUtils))
#print(len(extraUtils))
#print(extraUtils)    

In [ ]:
ip_recom = [1 if x>0 else 0 for x in PTNdata_master[(PTNdata_master['Student ID'] == userID) & (PTNdata_master['Training Exercise'] == TE)].iloc[0,2:-4].values]
print(ip_recom)

#percent similar to IPs recommendation.
#This is not really a valid metric right now 
corr = sum([True if i == j else False for i,j in zip(recom, ip_recom)])/numManeuvers
print('percent similar to IPs recommendation:',corr)

In [ ]:
#create list of extra maneuvers
for index,val in enumerate(extra):
    if val == 0:
        extra[index] = maneuverMIF_df.columns[index]
               
#filter out any maneuvers in recommendation
extra = list(filter((1).__ne__, extra))
#print(len(extra))
#print(extra)

#### extra_recom is an ordered list of next-best recommendations based on utilities.

In [ ]:
#zip the two lists.
#create a new, sorted list based on the zip using sorted().
#using a list comprehension extract the first elements of each pair from the sorted, zipped list.

extra_recom = [x for _,x in sorted(zip(extraUtils,extra),reverse= True)]
print('extra recomendations in order base on utility are as follows:')
print(extra_recom)

#### Maneuvers in Final Recommendation

In [ ]:
#define all manevuers in recommendation
for index,val in enumerate(recom):
    if val == 1:
        recom[index] = maneuverMIF_df.columns[index]
    else: recom[index] = 0
        
#filter out any maneuvers not in recommendation
final_recom = list(filter((0).__ne__, recom))

In [ ]:
print('The following',len(final_recom),'graded events are recommended on training exercise', TE, 'for student with ID:', userID)
#print(final_recom)
for x in final_recom:
    print(x)
    if x in fourShipFormationNames:
        print(x)

In [ ]:
#defining class to output bold text
class color:
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
#title
print(color.BOLD + color.UNDERLINE + "Event Count by Category:" + color.END)

#counts from recommendation
print('Basic:',sum(x in final_recom for x in basicManeuverNames))
print('Patterns:',sum(x in final_recom for x in patternsNames))
print('Contact:',sum(x in final_recom for x in contactNames))
print('Instrument:',sum(x in final_recom for x in instrumentNames))
print('Basic Formation:',sum(x in final_recom for x in basicFormationNames))
print('Tactical Formation:',sum(x in final_recom for x in tacticalFormationNames))
print('Low Level:',sum(x in final_recom for x in lowLevelNames))
print('4 Ship Formation:',sum(x in final_recom for x in fourShipFormationNames))
print('CAF Intro:',sum(x in final_recom for x in cafIntroNames))
print('MAF Intro:',sum(x in final_recom for x in mafIntroNames))

In [ ]:
### Total Time to make recommendation

In [ ]:
recommendTime = timeit.default_timer() - recommendTime_start
print('Time to generate recomendation', recommendTime, ' seconds')

In [ ]:
#error message that recommendation and extra dont add up 
assert len(final_recom)+len(extra_recom) == numManeuvers, "ERROR: You're recomendation is mixing with your extra maneuvers!"